<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/TwADR_bigtweet_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-9.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-9.validation.csv",header=None)
valid.head()

,0,1
0,13,up all night
1,13,insomnia
2,60,cigarette withdrawal
3,85,tremor
4,121,itches


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-9.test.csv",header=None)
test.head()

,0,1
0,62,feeling proper psycho
1,74,useless without
2,85,tremor
3,91,jitters
4,97,screwing with my short term memory


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-bigtweet.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'to',
 'the',
 'in',
 'rt',
 'of',
 'for',
 'a',
 'you',
 'i',
 'on',
 'and',
 'is',
 'disease',
 'pain',
 'my',
 'mood',
 'me',
 'increased',
 'amp',
 'with',
 'gluten',
 'disorder',
 ',',
 'abnormal',
 'withdrawal',
 'anxiety',
 'your',
 'free',
 'have',
 'it',
 'weight',
 'symptoms',
 'this',
 'out',
 'are',
 'decreased',
 'new',
 'disorders',
 'like',
 'from',
 'do',
 'about',
 'our',
 'glutenfree',
 'nt',
 '(',
 ')',
 'not',
 'up',
 'what',
 '-',
 'sleeplessness',
 'be',
 'behavior',
 'celiac',
 'syndrome',
 'drug',
 'got',
 'feeling',
 'at',
 'now',
 'diseases',
 'or',
 'somnolence',
 'insomnia',
 'sleep',
 'that',
 'disturbance',
 'blood',
 'finding',
 'so',
 'energy',
 'mental',
 'all',
 'food',
 'how',
 'we',
 'skin',
 'right',
 'depression',
 'can',
 'feel',
 'more',
 'gain',
 'via',
 'acute',
 'appetite',
 'if',
 'its',
 'tip',
 'body',
 'irritable',
 'ceiling',
 'feelings',
 'diet',
 

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.385629,3.484433,0.457799,09:28


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.296715,2.966052,0.516364,09:28


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.045030,2.918965,0.521931,09:30


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.982490,2.880805,0.526409,09:26


In [25]:
learn.save_encoder('TwADR-bigtweet_fold9_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-bigtweet_fold9_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.155895,6.270933,0.075000,00:26


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.293078,5.116701,0.183333,00:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.807323,4.914063,0.200000,01:04


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.487289,4.591508,0.241667,01:04
1,5.277765,4.418663,0.250000,01:09


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.031144,4.239302,0.266667,01:05
1,4.848877,4.053729,0.316667,01:10


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.632229,4.074724,0.308333,01:04
1,4.460736,3.912327,0.341667,01:11


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.242550,3.748682,0.366667,01:08
1,4.071370,3.696563,0.341667,01:12


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.896702,3.686009,0.358333,01:06
1,3.770431,3.617253,0.408333,01:08


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.624521,3.580287,0.358333,01:06
1,3.519294,3.628776,0.383333,01:03
2,3.323072,3.504692,0.408333,01:07
3,3.142925,3.488447,0.400000,01:09


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.994019,3.449880,0.408333,01:10
1,2.954769,3.400446,0.408333,01:09
2,2.795955,3.409014,0.425000,01:07
3,2.619745,3.392304,0.408333,01:05


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.430861,3.459643,0.433333,01:07
1,2.403485,3.386501,0.408333,01:06
2,2.264062,3.523529,0.416667,01:03
3,2.105603,3.437696,0.416667,01:08


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.992726,3.444021,0.441667,01:08
1,1.982725,3.550498,0.408333,01:06
2,1.842712,3.581520,0.441667,01:04
3,1.694969,3.494148,0.433333,01:08


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.590047,3.491974,0.425000,01:08
1,1.570721,3.741104,0.416667,01:07
2,1.445238,3.596417,0.416667,01:04
3,1.318542,3.540857,0.416667,01:08


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.230241,3.655508,0.425000,01:02
1,1.262104,3.780822,0.433333,01:06
2,1.177107,3.702451,0.441667,01:05
3,1.059323,3.709318,0.425000,01:07


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.970865,3.789827,0.433333,01:11
1,1.006660,3.798461,0.433333,01:03
2,0.932177,3.868465,0.425000,01:10
3,0.872720,3.925463,0.441667,01:03


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.817352,3.949486,0.425000,01:04
1,0.788994,4.021411,0.425000,01:06


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.746310,4.019054,0.425000,01:08
1,0.744744,4.082156,0.425000,01:04


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.682438,4.089408,0.433333,01:09


In [44]:
# save the best model

learn.save_encoder('TwADR-bigtweet_fold9')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.672192,4.196070,0.433333,01:09


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.660625,4.197916,0.433333,01:06


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.607686,4.406394,0.400000,01:02


In [51]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.618425,4.350301,0.416667,01:08


# Part three: Predict on the test dataset

In [52]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [53]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[1529, 61, 1204, 1099, 2113, 1295, 1295, 189, 582, 224, 224, 1099, 2147, 1936, 1936, 2147, 249, 813, 301, 1576, 1769, 1099, 1373, 749, 749, 24, 1815, 791, 1597, 792, 154, 978, 1308, 978, 1884, 249, 824, 852, 2023, 813, 1136, 974, 824, 974, 974, 109, 978, 978, 989, 1019, 1529, 1062, 1727, 1096, 1099, 2032, 1925, 1136, 1136, 1594, 1798, 2104, 1265, 1529, 2066, 947, 1319, 1099, 1099, 1356, 1099, 452, 1915, 1432, 857, 825, 1507, 1373, 1508, 1595, 1597, 1597, 1597, 1597, 744, 392, 824, 1610, 1099, 1526, 1701, 1099, 1373, 1526, 1125, 1711, 1787, 1718, 1718, 1373, 1718, 818, 1669, 452, 1727, 1787, 1787, 1787, 1787, 1787, 1787, 1787, 2194, 1718, 1718, 1531, 824, 852, 1099, 1529, 1942, 1099, 2024, 1787, 2032, 1787, 1787, 2074, 2103, 813, 2147, 2147, 2147, 2147, 1099, 2103, 1521, 2194, 1529, 1070, 2194, 2194]
[62, 74, 85, 91, 97, 189, 189, 189, 224, 224, 224, 233, 233, 233, 233, 233, 249, 250, 301, 504, 561, 620, 739, 749, 749, 749, 791, 791, 791, 792, 813, 818, 822, 822, 824, 824, 825, 

In [54]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

55
0.3873239436619718
